In [11]:
# Setup and imports
import re
import os
from pathlib import Path

import numpy as np
import mne
# from IPython.display import clear_output

os.chdir("/Users/yeganeh/Codes/MovieEEG-SourcePipeline/")

In [8]:
# Config
# -----------------------------
SFREQ = 512
DATA_DIR = Path("data/epochs")
SUBJECTS_DIR = Path("data")          # contains fsaverage/ (i.e., data/fsaverage)
FS_SUBJECT = "fsaverage"             # we use fsaverage anatomy for everyone

# Use an ico-4 source space (coarse; appropriate for Yeo-7 parcellation)
FS_SRC_FNAME = SUBJECTS_DIR / FS_SUBJECT / "bem" / "fsaverage-ico-4-src.fif"
FS_BEM_FNAME = SUBJECTS_DIR / FS_SUBJECT / "bem" / "fsaverage-5120-5120-5120-bem-sol.fif"

MONTAGE_NAME = "standard_1020"
os.environ["SUBJECTS_DIR"] = str(SUBJECTS_DIR)  # a temporary solution to make sure mne can find SUBJECTS_DIR

In [9]:
# Forward / Inverse builders
# -----------------------------
def _load_epochs(fname: Path) -> mne.Epochs:
    """Load epochs, ensure sampling rate and montage are set consistently."""
    epochs = mne.read_epochs(fname, preload=True, verbose=False)

    # Resample only if needed (avoids unnecessary filtering twice)
    if epochs.info["sfreq"] != SFREQ:
        epochs.resample(SFREQ, npad="auto", verbose=False)

    # montage = mne.channels.make_standard_montage(MONTAGE_NAME)
    # epochs.set_montage(montage, on_missing="warn")

    # Average reference as projector (consistent with MNE EEG workflows)
    epochs.set_eeg_reference("average", projection=True, verbose=False)
    epochs.apply_proj(verbose=False)

    return epochs


def make_forward(example_epochs_fname: Path) -> mne.Forward:
    """
    Build a single fsaverage forward solution to reuse across all subjects.
    Uses an example epochs file to define channel set and measurement info.
    """
    epochs = _load_epochs(example_epochs_fname)

    # fsaverage transform (built-in)
    trans = FS_SUBJECT
    src = mne.read_source_spaces(FS_SRC_FNAME, verbose=False)
    bem = mne.read_bem_solution(FS_BEM_FNAME, verbose=False)

    # Forward solution
    fwd = mne.make_forward_solution(
        info=epochs.info,
        trans=trans,
        src=src,
        bem=bem,
        eeg=True,
        mindist=5.0,
        n_jobs=-1,
        verbose=False,
    )
    return fwd


def make_inverse_from_baseline(
    baseline_epochs: mne.Epochs,
    fwd: mne.Forward,
) -> mne.minimum_norm.InverseOperator:
    """
    Build an inverse operator using baseline epochs to estimate noise covariance.
    Critically: covariance is computed directly from epochs (no pseudo-continuous stitching).
    """

    # Noise covariance from baseline epochs
    # 'shrunk' is stable for EEG; rank='info' respects projections
    cov = mne.compute_covariance(
        baseline_epochs,
        method="shrunk",
        rank="info",
        verbose=False,
    )

    inv = mne.minimum_norm.make_inverse_operator(
        info=baseline_epochs.info,
        forward=fwd,
        noise_cov=cov,
        loose=0.2,   # common for cortical source models
        depth=0.8,   # typical depth weighting
        verbose=False,
    )
    return inv

# Source -> labels
# -----------------------------
def extract_label_time_series(
    epochs: mne.Epochs,
    inv: mne.minimum_norm.InverseOperator,
    atlas_labels: list,
) -> np.ndarray:
    """
    Apply inverse on cut-locked epochs and extract parcel time courses.
    Returns: array of shape (n_epochs, n_labels, n_times)
    """
    stcs = mne.minimum_norm.apply_inverse_epochs(
        epochs,
        inverse_operator=inv,
        method="eLORETA",
        lambda2=1.0 / 9.0,
        pick_ori="normal",     # explicit orientation choice
        return_generator=False,
        verbose=False,
    )

    label_ts = mne.extract_label_time_course(
        stcs,
        labels=atlas_labels,
        src=inv["src"],
        mode="pca_flip",       # avoids sign cancellation; good for connectivity/ERP
        return_generator=False,
        verbose=False,
    )
    # label_ts: (n_epochs, n_labels, n_times)
    return label_ts

In [22]:
# Atlas labels (fsaverage annotations)
atlas_labels = mne.read_labels_from_annot(
    subject=FS_SUBJECT,
    parc="Yeo2011_7Networks_N1000",
    subjects_dir=str(SUBJECTS_DIR),
    verbose=False,
)

# Pick a single example file to define channel set for forward model
example = next(DATA_DIR.glob("*_art_nl_epo.fif"))
fwd = make_forward(example)

inv_cache = {}  # subject -> inverse operator built from baseline1

for epochs_path in sorted(DATA_DIR.glob("*_epo.fif")):
    m = re.search(r"^(\d+)_([^_]+_[^_]+)_epo$", epochs_path.stem)
    if m is None:
        continue
    subject, film = m.groups()
    epochs = _load_epochs(epochs_path)

    out_dir = Path("data/labels")
    out_dir.mkdir(parents=True, exist_ok=True)
    output_path_labels = out_dir / f"{subject}_{film}_labels.npz"

    if output_path_labels.exists():
        continue

    # Ensure we have an inverse per subject (from baseline1)
    if subject not in inv_cache:
        # pick baseline portion (avoid immediate pre-cut if you worry about anticipatory activity)
        epochs_base = epochs.copy().crop(tmin=-0.2, tmax=-0.05)
        inv_cache[subject] = make_inverse_from_baseline(epochs_base, fwd)

    inv = inv_cache[subject]

    print(f">>>>>>>> {subject} {film}")
    label_ts = extract_label_time_series(epochs, inv, atlas_labels)
    np.savez_compressed(output_path_labels, labels=label_ts)

/var/folders/6s/d689xnpd4553fw2yp02b3gv40000gn/T/ipykernel_28012/2831236813.py:32: RuntimeWarning: tmin is not in time interval. tmin is set to <class 'mne.epochs.EpochsFIF'>.tmin (-0.199219 s)
  epochs_base = epochs.copy().crop(tmin=-0.2, tmax=-0.05)


>>>>>>>> 01 art_nl
